In [20]:
# import des modules nécessaires
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import matplotlib.patches as mpatches
from distinctipy import distinctipy
from matplotlib.font_manager import FontProperties
from matplotlib import rc
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import clear_output, display
from scipy.stats import chi2_contingency as chi2_contingency
import statsmodels.formula.api as statsapi
import statsmodels as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import xlrd as xlrd
from matplotlib import font_manager # for mac users
import platform
import os
from sklearn.decomposition import PCA

# Modifier les paramètres d'affichage de la sortie des commandes dans Jupyter
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

# Pour lister les polices présentes
#fpaths = matplotlib.font_manager.findSystemFonts()
#for i in fpaths:
#    f = matplotlib.font_manager.get_font(i)
#    print(f.family_name)
# import de la police Garamond

system = platform.system()
print(system)
if system == 'Darwin':  # macOS
  font_path = '/Library/Fonts/GaramondPremrPro.otf'
elif system == 'Windows':
  font_path = 'C:/Windows/Fonts/Garamond.TTF'
else:  #  unix
  font_path = 'GARA.TTF'
print(font_path)

garamond_font = FontProperties(fname="/Library/Fonts/GaramondPremrPro.otf",size=14) # for MAC
garamond_font = FontProperties(fname="/Library/Fonts/Garamond.otf",size=14) # for WINDOWS

#    timesnewroman_font = FontProperties(fname="C:/Windows/Fonts/TIMES.TTF",size=14)

font_manager.findSystemFonts(fontpaths=font_path, fontext="ttf")
font_manager.findfont("Garamond") 

# Définissez la police par défaut pour toute la figure
matplotlib.rcParams['font.family'] = 'Garamond'

Windows
C:/Windows/Fonts/Garamond.TTF


In [21]:
# Chargement des données PINGOUIN
df_indian_leaver_patient = pd.read_table("input/mlsupervise/indian_liver_patient.csv",sep=",")

features_names = [
                    "Age of the patient",
                    "Gender of the patient",
                    "Total Bilirubin",
                    "Direct Bilirubin",
                    "Alkaline Phosphotase",
                    "Alamine Aminotransferase",
                    "Aspartate Aminotransferase",
                    "Total Protiens",
                    "Albumin",
                    "Albumin and Globulin Ratio"
                ]
class_names = ["Dataset"]

# Ajout titre colonnes
df_indian_leaver_patient.columns=features_names + class_names
#print(df_indian_leaver_patient.head(5))

df_indian_leaver_patient.shape

# modification Genre M = 1, Genre F = 0 pour n'avoir que des int ou des float dans les features

df_indian_leaver_patient["Gender of the patient"].replace(["Male","Female"],[1,0], inplace=True)
df_indian_leaver_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age of the patient          583 non-null    int64  
 1   Gender of the patient       583 non-null    int64  
 2   Total Bilirubin             583 non-null    float64
 3   Direct Bilirubin            583 non-null    float64
 4   Alkaline Phosphotase        583 non-null    int64  
 5   Alamine Aminotransferase    583 non-null    int64  
 6   Aspartate Aminotransferase  583 non-null    int64  
 7   Total Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin and Globulin Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(6)
memory usage: 50.2 KB


In [22]:
# Jeu d'entrainement, 80% des données

# import des librairies
from sklearn.model_selection import train_test_split
y=pd.DataFrame()
y["Dataset"]=df_indian_leaver_patient["Dataset"]
print("y :\n", y.head(5))
X=df_indian_leaver_patient.drop(columns={"Dataset"})
print("X:\n",X.iloc[:,0:7].head(5))


# Split du jeu de données
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1, shuffle=False, stratify=None)

print("X_train ", X_train.shape)
print("X_test ", X_test.shape)
print("y_train ", y_train.shape)
print("y_test ", y_test.shape)

y :
    Dataset
0        1
1        1
2        1
3        1
4        1
X:
    Age of the patient  Gender of the patient  Total Bilirubin  Direct Bilirubin  \
0                  65                      0              0.7               0.1   
1                  62                      1             10.9               5.5   
2                  62                      1              7.3               4.1   
3                  58                      1              1.0               0.4   
4                  72                      1              3.9               2.0   

   Alkaline Phosphotase  Alamine Aminotransferase  Aspartate Aminotransferase  
0                   187                        16                          18  
1                   699                        64                         100  
2                   490                        60                          68  
3                   182                        14                          20  
4                   195   